 # Evaluation of EAT

In [1]:
import argparse
from pathlib import Path
import yaml

# print current working directory
import os
import torch
import numpy as np
from torch.utils.data import DataLoader
from utils.helper_funcs import accuracy, count_parameters, mAP, measure_inference_time
print(os.getcwd())
# change path to the root of the project
os.chdir('/workspace/')

model_path = 'outputs/r1/best_model.pt'
args_path = 'outputs/r1/args.yml'
with (Path(args_path)).open() as f:
    args = yaml.load(f, Loader=yaml.Loader)
try:
    args = vars(args)
except:
    if 'net' in args.keys():
        del args['net']
    args_orig = args
    args = {}
    for k, v in args_orig.items():
        if isinstance(v, dict):
            for kk, vv in v.items():
                args[kk] = vv
        else:
            args[k] = v
args['f_res'] = Path('outputs/r1')
args['add_noise'] = None

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    


/


In [2]:
#######################
# Load PyTorch Models #
#######################
f_res = args['f_res']
from modules.soundnet import SoundNetRaw as SoundNet
ds_fac = np.prod(np.array(args['ds_factors'])) * 4
net = SoundNet(nf=args['nf'],
               dim_feedforward=args['dim_feedforward'],
               clip_length=args['seq_len'] // ds_fac,
               embed_dim=args['emb_dim'],
               n_layers=args['n_layers'],
               nhead=args['n_head'],
               n_classes=args['n_classes'],
               factors=args['ds_factors'],
               )
print('***********************************************')
print("#params: {}M".format(count_parameters(net)/1e6))
if torch.cuda.is_available() and device == torch.device("cuda"):
    t_b1 = measure_inference_time(net, torch.randn(1, 1, args['seq_len']))[0]
    print('inference time batch=1: {:.2f}[ms]'.format(t_b1))
    # t_b32 = measure_inference_time(net, torch.randn(32, 1, args['seq_len']))[0]
    # print('inference time batch=32: {:.2f}[ms]'.format(t_b32))
    print('***********************************************')
if (Path(model_path)).is_file():
    model = torch.load(model_path, map_location=torch.device(device))
    # model = chkpnt['model_dict']
else:
    raise ValueError
if 'use_dp' in args.keys() and args['use_dp']:
    from collections import OrderedDict
    state_dict = OrderedDict()
    for k, v in model.items():
        name = k.replace('module.', '')
        state_dict[name] = v
    net.load_state_dict(state_dict, strict=True)
else:
    net.load_state_dict(model, strict=True)
net.to(device)
if torch.cuda.device_count() > 1:
    from utils.helper_funcs import parse_gpu_ids
    args['gpu_ids'] = [i for i in range(torch.cuda.device_count())]
    net = torch.nn.DataParallel(net, device_ids=args['gpu_ids'])
    net.to('cuda:0')
net.eval()

***********************************************
#params: 5.183618M
torch.Size([1, 1, 114688])
inference time batch=1: 4.52[ms]
***********************************************


DataParallel(
  (module): SoundNetRaw(
    (start): Sequential(
      (0): ReflectionPad1d((3, 3))
      (1): Conv1d(1, 16, kernel_size=(7,), stride=(1,), bias=False)
      (2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (down): Sequential(
      (0): Down(
        (down): Sequential(
          (0): ReflectionPad1d((2, 2))
          (1): Conv1d(16, 32, kernel_size=(5,), stride=(1,), bias=False)
          (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (3): LeakyReLU(negative_slope=0.2, inplace=True)
          (4): AADownsample()
        )
      )
      (1): ResBlock1dTF(
        (block_t): Sequential(
          (0): ReflectionPad1d((7, 7))
          (1): Conv1d(32, 32, kernel_size=(15,), stride=(1,), groups=32, bias=False)
          (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (3): LeakyReL

## Load datasets

In [3]:
from datasets.esc_dataset import ESCDataset as SoundDataset
data_set = SoundDataset(args['data_path'],
                        mode='test',
                        segment_length=args['seq_len'],
                        sampling_rate=args['sampling_rate'],
                        fold_id=args['fold_id'],
                        transforms=None)

In [5]:
# number of class in dataset    
data_set.n_classes

AttributeError: 'ESCDataset' object has no attribute 'n_classes'

In [4]:
data_loader = DataLoader(data_set,
                         batch_size=128,
                         num_workers=8,
                         pin_memory=True if torch.cuda.is_available() else False,
                         shuffle=False)
labels = torch.zeros(len(data_loader.dataset), dtype=torch.float32).float()
preds = torch.zeros(len(data_loader.dataset), args['n_classes'], dtype=torch.float32).float()
# confusion_matrix = torch.zeros(args['n_classes'], args['n_classes'], dtype=torch.int)
confusion_matrix = torch.zeros(args['n_classes'], args['n_classes'], dtype=torch.int)
idx_start = 0
for i, (x, y) in enumerate(data_loader):
    with torch.no_grad():
        x = x.to(device)
        y = y.to(device)
        pred = net(x)
        _, y_est = torch.max(pred, 1)
        idx_end = idx_start + y.shape[0]
        preds[idx_start:idx_end, :] = pred
        labels[idx_start:idx_end] = y
        for t, p in zip(y.view(-1), y_est.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1
        print("{}/{}".format(i, len(data_loader)))
    idx_start = idx_end
acc_av = accuracy(preds.detach(), labels.detach(), [1, ])[0]
res = {
    "acc": acc_av,
    "preds": preds,
    "labels": labels.view(-1),
    "confusion_matrix": confusion_matrix
}
torch.save(res, args['f_res'] / "res.pt")
print("acc:{}".format(np.round(acc_av*100)/100))
print("cm:{}".format(confusion_matrix.diag().sum() / len(data_loader.dataset)))
print('***************************************')
bad_labels = []
for i, c in enumerate(confusion_matrix):
    i_est = c.argmax(-1)
    if i != i_est:
        # print('{} {} {}-->{}'.format(i, i_est.item(), data_set.labels[i], data_set.labels[i_est]))
        bad_labels.append([i, i_est])
print(bad_labels)

0/4
1/4
2/4
3/4
acc:35.25
cm:0.35249999165534973
***************************************
[[1, tensor(0)], [2, tensor(0)], [3, tensor(0)], [4, tensor(0)], [5, tensor(0)], [6, tensor(0)], [7, tensor(0)], [8, tensor(0)], [9, tensor(0)], [10, tensor(0)], [11, tensor(0)], [12, tensor(0)], [13, tensor(0)], [14, tensor(0)], [15, tensor(0)], [16, tensor(0)], [17, tensor(0)], [18, tensor(0)], [19, tensor(0)], [20, tensor(0)], [21, tensor(0)], [22, tensor(0)], [23, tensor(0)], [24, tensor(0)], [25, tensor(0)], [26, tensor(0)], [27, tensor(0)], [28, tensor(0)], [29, tensor(0)], [30, tensor(0)], [31, tensor(0)], [32, tensor(0)], [33, tensor(0)], [34, tensor(0)], [35, tensor(0)], [36, tensor(0)], [37, tensor(0)], [38, tensor(0)], [39, tensor(0)], [40, tensor(0)], [41, tensor(0)], [42, tensor(0)], [43, tensor(0)], [44, tensor(0)], [45, tensor(0)], [46, tensor(0)], [47, tensor(0)], [48, tensor(0)], [49, tensor(0)]]


In [21]:
# get random sample from train_set
import random
index = random.randint(0, len(train_set))
sample = train_set[index]

In [22]:
sample[0].shape, sample[1]

(torch.Size([1, 114688]), 0)